In [ ]:
import os
import re
import glob
import numpy as np
import PIL.Image as img
import random
from sklearn import metrics
from sklearn.utils.linear_assignment_ import linear_assignment
import tensorflow as tf

/usr/local/lib/python3.6/dist-packages/sklearn/utils/linear_assignment_.py:22: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


In [ ]:
# Option
mode = 'Training'
num_cluster = 10
eps = 1e-10
height = 28
width = 28
channel = 1

In [ ]:
# Get Datas and Labels as batch_size
def get_batch(batch_size, img_data, imgt_labels):
    batch_index = random.sample(range(len(imgt_labels)), batch_size)

    batch_data = np.empty([batch_size, height, width, channel], dtype=np.float32)
    batch_label = np.empty([batch_size], dtype=np.int32)

    for n, i in enumerate(batch_index):
        batch_data[n, ...] = img_data[i, ...]
        batch_label[n] = imgt_labels[i]
    return batch_data, batch_label

In [ ]:
# Get Datas and Labels as batch_size for Testing
def get_batch_test(batch_size, img_data, i):
    batch_data = np.copy(img_data[batch_size * i:batch_size * (i + 1), ...])
    return batch_data


In [ ]:
def clustering_acc(y_true, y_pred):
    y_true = y_true.astype(np.int64)
    assert y_pred.size == y_true.size
    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1
    ind = linear_assignment(w.max() - w)

    return sum([w[i, j] for i, j in ind]) * 1.0 / y_pred.size

In [ ]:

def NMI(y_true, y_pred):
    return metrics.normalized_mutual_info_score(y_true, y_pred)

def ARI(y_true, y_pred):
    return metrics.adjusted_rand_score(y_true, y_pred)

In [ ]:
from tensorflow.python.keras import backend as k

In [ ]:
tf.compat.v1.get_default_graph()

In [ ]:
def ConvNetwork(in_img, num_cluster, name='ConvNetwork', reuse=False):
	with tf.variable_scope(name, reuse=reuse):
		# conv1
		conv1 = tf.layers.conv2d(in_img, 64, [3,3], [1,1], padding='valid', activation=None, kernel_initializer=tf.keras.initializers.he_normal())
		conv1 = tf.layers.batch_normalization(conv1, axis=-1, epsilon=1e-5, training=True, trainable=False)
		conv1 = tf.nn.relu(conv1)
		# conv2
		conv2 = tf.layers.conv2d(conv1, 64, [3,3], [1,1], padding='valid', activation=None, kernel_initializer=tf.keras.initializers.he_normal())
		conv2 = tf.layers.batch_normalization(conv2, axis=-1, epsilon=1e-5, training=True, trainable=False)
		conv2 = tf.nn.relu(conv2)
		# conv3
		conv3 = tf.layers.conv2d(conv2, 64, [3,3], [1,1], padding='valid', activation=None, kernel_initializer=tf.keras.initializers.he_normal())
		conv3 = tf.layers.batch_normalization(conv3, axis=-1, epsilon=1e-5, training=True, trainable=False)
		conv3 = tf.nn.relu(conv3)
		conv3 = tf.layers.max_pooling2d(conv3, [2,2], [2,2])
		conv3 = tf.layers.batch_normalization(conv3, axis=-1, epsilon=1e-5, training=True, trainable=False)
		# conv4
		conv4 = tf.layers.conv2d(conv3, 128, [3,3], [1,1], padding='valid', activation=None, kernel_initializer=tf.keras.initializers.he_normal())
		conv4 = tf.layers.batch_normalization(conv4, axis=-1, epsilon=1e-5, training=True, trainable=False)
		conv4 = tf.nn.relu(conv4)
		# conv5
		conv5 = tf.layers.conv2d(conv4, 128, [3,3], [1,1], padding='valid', activation=None, kernel_initializer=tf.keras.initializers.he_normal())
		conv5 = tf.layers.batch_normalization(conv5, axis=-1, epsilon=1e-5, training=True, trainable=False)
		conv5 = tf.nn.relu(conv5)
		# conv6
		conv6 = tf.layers.conv2d(conv5, 128, [3,3], [1,1], padding='valid', activation=None, kernel_initializer=tf.keras.initializers.he_normal())
		conv6 = tf.layers.batch_normalization(conv6, axis=-1, epsilon=1e-5, training=True, trainable=False)
		conv6 = tf.nn.relu(conv6)
		conv6 = tf.layers.max_pooling2d(conv6, [2,2], [2,2])
		conv6 = tf.layers.batch_normalization(conv6, axis=-1, epsilon=1e-5, training=True, trainable=False)
		# conv7
		conv7 = tf.layers.conv2d(conv6, 10, [1,1], [1,1], padding='valid', activation=None, kernel_initializer=tf.keras.initializers.he_normal())
		conv7 = tf.layers.batch_normalization(conv7, axis=-1, epsilon=1e-5, training=True, trainable=False)
		conv7 = tf.nn.relu(conv7)
		conv7 = tf.layers.average_pooling2d(conv7, [2,2], [2,2])
		conv7 = tf.layers.batch_normalization(conv7, axis=-1, epsilon=1e-5, training=True, trainable=False)
		conv7_flat = tf.layers.flatten(conv7)

		# dense8
		fc8 = tf.layers.dense(conv7_flat, 10, kernel_initializer=tf.initializers.identity())
		fc8 = tf.layers.batch_normalization(fc8, axis=-1, epsilon=1e-5, training=True, trainable=False)
		fc8 = tf.nn.relu(fc8)
		# dense9
		fc9 = tf.layers.dense(fc8, num_cluster, kernel_initializer=tf.initializers.identity())
		fc9 = tf.layers.batch_normalization(fc9, axis=-1, epsilon=1e-5, training=True, trainable=False)
		fc9 = tf.nn.relu(fc9)

		out = tf.nn.softmax(fc9)

	return out


In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
image_pool_input = tf.placeholder(shape=[None, height, width, channel], dtype=tf.float32, name='image_pool_input')
u_thres = tf.placeholder(shape=[], dtype=tf.float32, name='u_thres')
l_thres = tf.placeholder(shape=[], dtype=tf.float32, name='l_thres')
lr = tf.placeholder(shape=[], dtype=tf.float32, name='learning_rate')

label_feat = ConvNetwork(image_pool_input, num_cluster, name='ConvNetwork', reuse=False)
label_feat_norm = tf.nn.l2_normalize(label_feat, dim=1)
sim_mat = tf.matmul(label_feat_norm, label_feat_norm, transpose_b=True)

pos_loc = tf.greater(sim_mat, u_thres, name='greater')
neg_loc = tf.less(sim_mat, l_thres, name='less')
pos_loc_mask = tf.cast(pos_loc, dtype=tf.float32)
neg_loc_mask = tf.cast(neg_loc, dtype=tf.float32)

pred_label = tf.argmax(label_feat, axis=1)

# Deep Adaptive Image Clustering Cost Function Optimize
pos_entropy = tf.multiply(-tf.log(tf.clip_by_value(sim_mat, eps, 1.0)), pos_loc_mask)
neg_entropy = tf.multiply(-tf.log(tf.clip_by_value(1-sim_mat, eps, 1.0)), neg_loc_mask)

loss_sum = tf.reduce_mean(pos_entropy) + tf.reduce_mean(neg_entropy)
train_op = tf.train.RMSPropOptimizer(lr).minimize(loss_sum)




Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
Instructions for updating:
Use keras.layers.AveragePooling2D instead.
Instructions for updating:
Use keras.layers.Flatten instead.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
dim is deprecated, use axis instead
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
import tensorflow as tf
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
mnist_train = np.reshape(train_images, (-1, 28, 28, 1))  # reshape into 1-channel image
mnist_train_labels = np.asarray(train_labels, dtype=np.int32)
mnist_test = np.reshape(test_images, (-1, 28, 28, 1))  # reshape into 1-channel image
mnist_test_labels = np.asarray(test_labels, dtype=np.int32)
image_data = np.concatenate([mnist_train, mnist_test], axis=0)
image_label = np.concatenate([mnist_train_labels, mnist_test_labels], axis=0)
print(len(image_data))

70000


In [ ]:
# Compress Dimension
mapping = {}
mapped_label = 0
for index,data in enumerate(image_label):
  if(data in mapping):
    image_label[index] = mapping[data]
  else:
    image_label[index] = mapping[data] = mapped_label+1
    mapped_label += 1

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [ ]:
saver = tf.train.Saver()

In [ ]:
if mode == 'Training':
    batch_size = 128
    test_batch_size = 512
    base_lr = 0.001
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        lamda = 0
        epoch = 1
        u = 0.95
        l = 0.455

        while u > l:
            u = 0.95 - lamda
            l = 0.455 + 0.1 * lamda
            print(u, l)
            for i in range(1, int(1001)):  # 1000 iterations is roughly 1 epoch
                data_samples, _ = get_batch(batch_size, image_data, image_label)
                feed_dict = {image_pool_input: data_samples, u_thres: u, l_thres: l, lr: base_lr}
                train_loss, _ = sess.run([loss_sum, train_op], feed_dict=feed_dict)
                if i % 20 == 0:
                    print('training loss at iter %d is %f' % (i, train_loss))

            lamda += 1.1 * 0.009
            print(lamda)
            # run testing every epoch
            data_samples, data_labels = get_batch(test_batch_size, image_data, image_label)
            feed_dict = {image_pool_input: data_samples}
            pred_cluster = sess.run(pred_label, feed_dict=feed_dict)

            acc = clustering_acc(data_labels, pred_cluster)
            nmi = NMI(data_labels, pred_cluster)
            ari = ARI(data_labels, pred_cluster)
            print('testing NMI, ARI, ACC at epoch %d is %f, %f, %f.' % (epoch, nmi, ari, acc))

            if epoch % 5 == 0:  # save model at every 5 epochs
                model_name = 'DAC_ep_' + str(epoch) + '.ckpt'
                save_path = saver.save(sess, 'DAC_models/' + model_name)
                print("Model saved in file: %s" % save_path)

            epoch += 1

elif mode == 'Testing':
    test_batch_size = 1000
    with tf.Session() as sess:
        saver.restore(sess, "DAC_models/DAC_ep_45.ckpt")
        print('model restored!')
        all_predictions = np.zeros([len(image_label)], dtype=np.float32)
        for i in range(len(image_datsa) // test_batch_size):
            data_samples = get_batch_test(test_batch_size, image_data, i)
            feed_dict = {image_pool_input: data_samples}
            pred_cluster = sess.run(pred_label, feed_dict=feed_dict)
            all_predictions[i * test_batch_size:(i + 1) * test_batch_size] = pred_cluster

        acc = clustering_acc(image_label.astype(int), all_predictions.astype(int))
        nmi = NMI(image_label.astype(int), all_predictions.astype(int))
        ari = ARI(image_label.astype(int), all_predictions.astype(int))
        print('testing NMI, ARI, ACC are %f, %f, %f.' % (nmi, ari, acc))

0.95 0.455
training loss at iter 20 is 0.149347
training loss at iter 40 is 0.170385
training loss at iter 60 is 0.146097
training loss at iter 80 is 0.178811
training loss at iter 100 is 0.183752
training loss at iter 120 is 0.175200
training loss at iter 140 is 0.185121
training loss at iter 160 is 0.177561
training loss at iter 180 is 0.180105
training loss at iter 200 is 0.179779
training loss at iter 220 is 0.171554
training loss at iter 240 is 0.162225
training loss at iter 260 is 0.164176
training loss at iter 280 is 0.174341
training loss at iter 300 is 0.157272
training loss at iter 320 is 0.151768
training loss at iter 340 is 0.156998
training loss at iter 360 is 0.142101
training loss at iter 380 is 0.146148
training loss at iter 400 is 0.153183
training loss at iter 420 is 0.140798
training loss at iter 440 is 0.157799
training loss at iter 460 is 0.151539
training loss at iter 480 is 0.143286
training loss at iter 500 is 0.162643
training loss at iter 520 is 0.149354
train

/usr/local/lib/python3.6/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


testing NMI, ARI, ACC at epoch 1 is 0.934073, 0.922542, 0.962891.
0.9400999999999999 0.45599
training loss at iter 20 is 0.138686
training loss at iter 40 is 0.130629
training loss at iter 60 is 0.137978
training loss at iter 80 is 0.133647
training loss at iter 100 is 0.140571
training loss at iter 120 is 0.132992
training loss at iter 140 is 0.138531
training loss at iter 160 is 0.132507
training loss at iter 180 is 0.137796
training loss at iter 200 is 0.128005
training loss at iter 220 is 0.136457
training loss at iter 240 is 0.148888
training loss at iter 260 is 0.143744
training loss at iter 280 is 0.143197
training loss at iter 300 is 0.132649
training loss at iter 320 is 0.129489
training loss at iter 340 is 0.135899
training loss at iter 360 is 0.129418
training loss at iter 380 is 0.137737
training loss at iter 400 is 0.138591
training loss at iter 420 is 0.129365
training loss at iter 440 is 0.134367
training loss at iter 460 is 0.133880
training loss at iter 480 is 0.139443

/usr/local/lib/python3.6/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


training loss at iter 20 is 0.128675
training loss at iter 40 is 0.138146
training loss at iter 60 is 0.135212
training loss at iter 80 is 0.133597
training loss at iter 100 is 0.139735
training loss at iter 120 is 0.136165
training loss at iter 140 is 0.129509
training loss at iter 160 is 0.133774
training loss at iter 180 is 0.154692
training loss at iter 200 is 0.124796
training loss at iter 220 is 0.126286
training loss at iter 240 is 0.135680
training loss at iter 260 is 0.134470
training loss at iter 280 is 0.136192
training loss at iter 300 is 0.132227
training loss at iter 320 is 0.125263
training loss at iter 340 is 0.131121
training loss at iter 360 is 0.161975
training loss at iter 380 is 0.129970
training loss at iter 400 is 0.137771
training loss at iter 420 is 0.127105
training loss at iter 440 is 0.145586
training loss at iter 460 is 0.135014
training loss at iter 480 is 0.131194
training loss at iter 500 is 0.127329
training loss at iter 520 is 0.131351
training loss at

...

/usr/local/lib/python3.6/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


training loss at iter 20 is 0.122273
training loss at iter 40 is 0.116654
training loss at iter 60 is 0.131121
training loss at iter 80 is 0.122498
training loss at iter 100 is 0.124764
training loss at iter 120 is 0.131157
training loss at iter 140 is 0.128475
training loss at iter 160 is 0.125961
training loss at iter 180 is 0.126790
training loss at iter 200 is 0.122757
training loss at iter 220 is 0.138188
training loss at iter 240 is 0.135761
training loss at iter 260 is 0.119640
training loss at iter 280 is 0.135086
training loss at iter 300 is 0.128450
training loss at iter 320 is 0.129770
training loss at iter 340 is 0.116304
training loss at iter 360 is 0.133214
training loss at iter 380 is 0.134833
training loss at iter 400 is 0.117580
training loss at iter 420 is 0.126938
training loss at iter 440 is 0.125587
training loss at iter 460 is 0.128665
training loss at iter 480 is 0.126112
training loss at iter 500 is 0.125144
training loss at iter 520 is 0.118971
training loss at

/usr/local/lib/python3.6/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


training loss at iter 20 is 0.140386
training loss at iter 40 is 0.123780
training loss at iter 60 is 0.122276
training loss at iter 80 is 0.120907
training loss at iter 100 is 0.133124
training loss at iter 120 is 0.121180
training loss at iter 140 is 0.128807
training loss at iter 160 is 0.124775
training loss at iter 180 is 0.123807
training loss at iter 200 is 0.120280
training loss at iter 220 is 0.125723
training loss at iter 240 is 0.117817
training loss at iter 260 is 0.116450
training loss at iter 280 is 0.118429
training loss at iter 300 is 0.127243
training loss at iter 320 is 0.130316
training loss at iter 340 is 0.141857
training loss at iter 360 is 0.126606
training loss at iter 380 is 0.124468
training loss at iter 400 is 0.126355
training loss at iter 420 is 0.125211
training loss at iter 440 is 0.120545
training loss at iter 460 is 0.119184
training loss at iter 480 is 0.132607
training loss at iter 500 is 0.127184
training loss at iter 520 is 0.128455
training loss at

/usr/local/lib/python3.6/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)
